# High-Level Reinforcement Learning for Unitree G1

This notebook implements a **high-level RL framework** for the Unitree G1 humanoid robot, focusing on:

1. **State Space:** Robot observations + USB controller commands
2. **Action Space:** Abstract high-level actions (path planning, gestures, audio)
3. **Reward Design:** Task-oriented rewards for real-world applications
4. **Training Pipeline:** From simulation to real robot deployment

## Key Concepts

**High-Level RL** operates at the **task planning level**, abstracting away low-level motor control:

```
Traditional RL:        State → Neural Network → Joint Torques
                       (Low-level, 29-DOF control)

High-Level RL:         State → Neural Network → Abstract Actions
                       (Task-level: "walk to X", "wave hand", "say hello")
                       
                       Abstract Actions → Motion Primitives → Joint Commands
```

**Benefits:**
- Faster learning (smaller action space)
- Better generalization
- Safer (primitives are pre-validated)
- Human-interpretable actions

---

## Table of Contents

1. [State Space Design](#state-space)
2. [USB Controller Integration](#controller)
3. [High-Level Action Space](#actions)
4. [Motion Primitives Library](#primitives)
5. [Reward Function Design](#rewards)
6. [Environment Implementation](#environment)
7. [Training with PPO](#training)
8. [Deployment to Real Robot](#deployment)
9. [Example Applications](#applications)

---

## Setup & Dependencies

In [ ]:
# Install dependencies
!pip install gymnasium stable-baselines3 torch numpy matplotlib
!pip install pygame  # For USB controller support
!pip install pynput  # Alternative: keyboard/mouse input

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass
from enum import Enum
import time
import json

# RL frameworks
import gymnasium as gym
from gymnasium import spaces
import torch
import torch.nn as nn

try:
    from stable_baselines3 import PPO
    from stable_baselines3.common.vec_env import DummyVecEnv
    from stable_baselines3.common.callbacks import BaseCallback
    SB3_AVAILABLE = True
except ImportError:
    print("Warning: stable-baselines3 not available")
    SB3_AVAILABLE = False

# USB Controller
try:
    import pygame
    PYGAME_AVAILABLE = True
except ImportError:
    print("Warning: pygame not available (USB controller input disabled)")
    PYGAME_AVAILABLE = False

# Unitree SDK
try:
    from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelPublisher, ChannelFactoryInitialize
    from unitree_sdk2py.idl.unitree_hg.msg.dds_ import LowCmd_, LowState_
    from unitree_sdk2py.g1.loco.g1_loco_client import LocoClient
    from unitree_sdk2py.g1.audio.g1_audio_client import AudioClient
    from unitree_sdk2py.g1.arm.g1_arm_action_client import G1ArmActionClient, action_map
    UNITREE_SDK_AVAILABLE = True
except ImportError:
    print("Warning: Unitree SDK not available (simulation only)")
    UNITREE_SDK_AVAILABLE = False

print("✓ Imports complete")
print(f"  - PyGame (Controller): {PYGAME_AVAILABLE}")
print(f"  - Stable-Baselines3: {SB3_AVAILABLE}")
print(f"  - Unitree SDK: {UNITREE_SDK_AVAILABLE}")

---
<a id='state-space'></a>
## 1. State Space Design

The state space combines **robot observations** and **human commands** from a USB controller.

### State Components

#### A. Robot Observations (Proprioceptive)
- **Base Orientation:** Roll, pitch, yaw (3D)
- **Base Angular Velocity:** ωx, ωy, ωz (3D)
- **Base Linear Velocity:** vx, vy, vz (3D)
- **Joint Positions:** 29 joint angles (29D)
- **Joint Velocities:** 29 joint velocities (29D)
- **Contact State:** Foot contact sensors (2D - left/right)

#### B. Robot Observations (Exteroceptive)
- **Goal Direction:** Relative vector to target (3D)
- **Goal Distance:** Scalar distance to target (1D)
- **Obstacle Proximity:** Nearest obstacle distance in 8 directions (8D)

#### C. Human Commands (USB Controller)
- **Velocity Command:** Desired (vx, vy, ω) from joystick (3D)
- **Mode Selector:** Discrete mode (walk/run/idle) (1D)
- **Gesture Trigger:** Which gesture to perform (1D)

**Total State Dimension:** 3 + 3 + 3 + 29 + 29 + 2 + 3 + 1 + 8 + 3 + 1 + 1 = **86 dimensions**

In [ ]:
@dataclass
class RobotState:
    """Complete state representation for high-level RL"""
    
    # Proprioceptive (internal sensors)
    base_orientation: np.ndarray  # [roll, pitch, yaw] in radians
    base_angular_velocity: np.ndarray  # [wx, wy, wz] in rad/s
    base_linear_velocity: np.ndarray  # [vx, vy, vz] in m/s
    joint_positions: np.ndarray  # 29 joint angles in radians
    joint_velocities: np.ndarray  # 29 joint velocities in rad/s
    foot_contacts: np.ndarray  # [left_contact, right_contact] (0 or 1)
    
    # Exteroceptive (external sensors)
    goal_direction: np.ndarray  # Unit vector pointing to goal [x, y, z]
    goal_distance: float  # Euclidean distance to goal in meters
    obstacle_proximity: np.ndarray  # Distance to nearest obstacle in 8 directions
    
    # Human commands (from USB controller)
    cmd_velocity: np.ndarray  # Desired [vx, vy, omega] from joystick
    cmd_mode: int  # 0=idle, 1=walk, 2=run
    cmd_gesture: int  # Gesture ID (0=none, 1=wave, 2=handshake, etc.)
    
    def to_array(self) -> np.ndarray:
        """Convert state to flat numpy array for RL"""
        return np.concatenate([
            self.base_orientation,
            self.base_angular_velocity,
            self.base_linear_velocity,
            self.joint_positions,
            self.joint_velocities,
            self.foot_contacts,
            self.goal_direction,
            [self.goal_distance],
            self.obstacle_proximity,
            self.cmd_velocity,
            [self.cmd_mode],
            [self.cmd_gesture]
        ])
    
    @property
    def dimension(self) -> int:
        """Total state dimension"""
        return len(self.to_array())

# Example state initialization
example_state = RobotState(
    base_orientation=np.zeros(3),
    base_angular_velocity=np.zeros(3),
    base_linear_velocity=np.zeros(3),
    joint_positions=np.zeros(29),
    joint_velocities=np.zeros(29),
    foot_contacts=np.array([1, 1]),  # Both feet on ground
    goal_direction=np.array([1, 0, 0]),  # Goal straight ahead
    goal_distance=5.0,  # 5 meters away
    obstacle_proximity=np.ones(8) * 10.0,  # No nearby obstacles
    cmd_velocity=np.zeros(3),
    cmd_mode=1,  # Walk mode
    cmd_gesture=0  # No gesture
)

print(f"State dimension: {example_state.dimension}")
print(f"State vector shape: {example_state.to_array().shape}")

---
<a id='controller'></a>
## 2. USB Controller Integration

We'll use a standard USB game controller (e.g., Xbox, PlayStation) to provide human commands.

### Controller Mapping

```
┌─────────────────────────────────────────┐
│        USB Game Controller              │
├─────────────────────────────────────────┤
│                                         │
│  Left Stick:  → Forward/Backward (vx)  │
│               → Strafe Left/Right (vy)  │
│                                         │
│  Right Stick: → Rotate Left/Right (ω)  │
│                                         │
│  Buttons:                               │
│    A/X     → Walk Mode                  │
│    B/Circle → Run Mode                   │
│    Y/Triangle → Idle/Stop                │
│    LB      → Wave Hand                  │
│    RB      → Handshake                  │
│    LT      → Clap                       │
│    Start   → Emergency Stop             │
└─────────────────────────────────────────┘
```

In [ ]:
class USBControllerInput:
    """Read commands from USB game controller (Xbox/PlayStation compatible)"""
    
    def __init__(self):
        self.available = False
        self.joystick = None
        
        if PYGAME_AVAILABLE:
            try:
                pygame.init()
                pygame.joystick.init()
                
                if pygame.joystick.get_count() > 0:
                    self.joystick = pygame.joystick.Joystick(0)
                    self.joystick.init()
                    self.available = True
                    print(f"✓ Controller connected: {self.joystick.get_name()}")
                    print(f"  Axes: {self.joystick.get_numaxes()}")
                    print(f"  Buttons: {self.joystick.get_numbuttons()}")
                else:
                    print("⚠️  No USB controller detected")
            except Exception as e:
                print(f"⚠️  Controller initialization failed: {e}")
        
        # Default command values
        self.cmd_velocity = np.zeros(3)
        self.cmd_mode = 0
        self.cmd_gesture = 0
        self.emergency_stop = False
    
    def read_commands(self) -> Dict[str, Any]:
        """
        Read current controller state
        
        Returns:
            commands: Dictionary with velocity, mode, gesture commands
        """
        if not self.available:
            # Return default commands if no controller
            return {
                'velocity': self.cmd_velocity,
                'mode': self.cmd_mode,
                'gesture': self.cmd_gesture,
                'emergency_stop': False
            }
        
        # Process pygame events
        pygame.event.pump()
        
        # Read axes (joysticks)
        # Left stick: axis 0 (left/right), axis 1 (up/down)
        # Right stick: axis 3 (left/right)
        left_x = self.joystick.get_axis(0) if self.joystick.get_numaxes() > 0 else 0.0
        left_y = -self.joystick.get_axis(1) if self.joystick.get_numaxes() > 1 else 0.0  # Invert Y
        right_x = self.joystick.get_axis(3) if self.joystick.get_numaxes() > 3 else 0.0
        
        # Apply deadzone (ignore small movements)
        deadzone = 0.15
        left_x = left_x if abs(left_x) > deadzone else 0.0
        left_y = left_y if abs(left_y) > deadzone else 0.0
        right_x = right_x if abs(right_x) > deadzone else 0.0
        
        # Map to velocity commands
        # Scale: joystick [-1, 1] → velocity [max_speed]
        max_linear = 0.5  # m/s
        max_angular = 0.8  # rad/s
        
        self.cmd_velocity = np.array([
            left_y * max_linear,   # Forward/backward
            left_x * max_linear,   # Strafe left/right
            right_x * max_angular  # Rotate
        ])
        
        # Read buttons
        button_a = self.joystick.get_button(0) if self.joystick.get_numbuttons() > 0 else 0
        button_b = self.joystick.get_button(1) if self.joystick.get_numbuttons() > 1 else 0
        button_y = self.joystick.get_button(3) if self.joystick.get_numbuttons() > 3 else 0
        button_lb = self.joystick.get_button(4) if self.joystick.get_numbuttons() > 4 else 0
        button_rb = self.joystick.get_button(5) if self.joystick.get_numbuttons() > 5 else 0
        button_lt = self.joystick.get_button(6) if self.joystick.get_numbuttons() > 6 else 0
        button_start = self.joystick.get_button(7) if self.joystick.get_numbuttons() > 7 else 0
        
        # Mode selection
        if button_a:
            self.cmd_mode = 1  # Walk
        elif button_b:
            self.cmd_mode = 2  # Run
        elif button_y:
            self.cmd_mode = 0  # Idle
        
        # Gesture selection
        if button_lb:
            self.cmd_gesture = 1  # Wave
        elif button_rb:
            self.cmd_gesture = 2  # Handshake
        elif button_lt:
            self.cmd_gesture = 3  # Clap
        else:
            self.cmd_gesture = 0  # None
        
        # Emergency stop
        self.emergency_stop = bool(button_start)
        
        return {
            'velocity': self.cmd_velocity.copy(),
            'mode': self.cmd_mode,
            'gesture': self.cmd_gesture,
            'emergency_stop': self.emergency_stop
        }
    
    def get_command_string(self) -> str:
        """Human-readable command description"""
        mode_names = ['IDLE', 'WALK', 'RUN']
        gesture_names = ['None', 'Wave', 'Handshake', 'Clap']
        
        return (
            f"Mode: {mode_names[self.cmd_mode]} | "
            f"Vel: [{self.cmd_velocity[0]:.2f}, {self.cmd_velocity[1]:.2f}, {self.cmd_velocity[2]:.2f}] | "
            f"Gesture: {gesture_names[self.cmd_gesture]}"
        )

# Test controller
controller = USBControllerInput()

if controller.available:
    print("\nTesting controller (move sticks and press buttons)...")
    for i in range(50):  # Read for 5 seconds
        commands = controller.read_commands()
        print(f"\r{controller.get_command_string()}", end='', flush=True)
        time.sleep(0.1)
    print("\n✓ Controller test complete")
else:
    print("\n⚠️  Controller not available - will use simulated commands")

---
<a id='actions'></a>
## 3. High-Level Action Space

Instead of directly controlling 29 joints, we define **high-level abstract actions**:

### Action Space Design

#### Discrete Actions (10 total)

| Action ID | Name | Description |
|-----------|------|-------------|
| 0 | `IDLE` | Stand still, damping mode |
| 1 | `WALK_FORWARD` | Walk forward at commanded velocity |
| 2 | `WALK_BACKWARD` | Walk backward |
| 3 | `STRAFE_LEFT` | Sidestep left |
| 4 | `STRAFE_RIGHT` | Sidestep right |
| 5 | `ROTATE_LEFT` | Turn left in place |
| 6 | `ROTATE_RIGHT` | Turn right in place |
| 7 | `GESTURE_WAVE` | Perform wave gesture |
| 8 | `GESTURE_HANDSHAKE` | Perform handshake gesture |
| 9 | `GESTURE_CLAP` | Perform clap gesture |

#### Continuous Parameters (3D)

- **Velocity Scale:** [0, 1] - How fast to execute action
- **Direction Bias:** [-1, 1] - Fine-tune direction
- **Audio Volume:** [0, 1] - Volume for audio feedback

**Total Action Space:** Discrete(10) + Box(3)

In [ ]:
class HighLevelAction(Enum):
    """Enumeration of high-level actions"""
    IDLE = 0
    WALK_FORWARD = 1
    WALK_BACKWARD = 2
    STRAFE_LEFT = 3
    STRAFE_RIGHT = 4
    ROTATE_LEFT = 5
    ROTATE_RIGHT = 6
    GESTURE_WAVE = 7
    GESTURE_HANDSHAKE = 8
    GESTURE_CLAP = 9

@dataclass
class ActionCommand:
    """Complete action specification"""
    action_type: HighLevelAction
    velocity_scale: float  # [0, 1]
    direction_bias: float  # [-1, 1]
    audio_volume: float  # [0, 1]
    
    def to_motion_command(self) -> Dict[str, Any]:
        """
        Convert high-level action to low-level motion command
        
        Returns:
            motion_cmd: Dictionary with target velocities and gesture flags
        """
        # Base velocities for each action type
        action_velocities = {
            HighLevelAction.IDLE: (0.0, 0.0, 0.0),
            HighLevelAction.WALK_FORWARD: (0.3, 0.0, 0.0),
            HighLevelAction.WALK_BACKWARD: (-0.2, 0.0, 0.0),
            HighLevelAction.STRAFE_LEFT: (0.0, 0.2, 0.0),
            HighLevelAction.STRAFE_RIGHT: (0.0, -0.2, 0.0),
            HighLevelAction.ROTATE_LEFT: (0.0, 0.0, 0.5),
            HighLevelAction.ROTATE_RIGHT: (0.0, 0.0, -0.5),
            HighLevelAction.GESTURE_WAVE: (0.0, 0.0, 0.0),
            HighLevelAction.GESTURE_HANDSHAKE: (0.0, 0.0, 0.0),
            HighLevelAction.GESTURE_CLAP: (0.0, 0.0, 0.0),
        }
        
        vx, vy, vw = action_velocities[self.action_type]
        
        # Apply velocity scaling
        vx *= self.velocity_scale
        vy *= self.velocity_scale
        vw *= self.velocity_scale
        
        # Apply direction bias (adds lateral component)
        vy += self.direction_bias * 0.1
        
        # Determine if gesture should be executed
        is_gesture = self.action_type.value >= 7
        gesture_id = self.action_type.value - 7 if is_gesture else None
        
        return {
            'velocity': np.array([vx, vy, vw]),
            'is_gesture': is_gesture,
            'gesture_id': gesture_id,
            'audio_volume': self.audio_volume
        }
    
    def get_description(self) -> str:
        """Human-readable action description"""
        return (
            f"{self.action_type.name} "
            f"(speed={self.velocity_scale:.2f}, "
            f"bias={self.direction_bias:.2f}, "
            f"volume={self.audio_volume:.2f})"
        )

# Example actions
example_actions = [
    ActionCommand(HighLevelAction.WALK_FORWARD, 0.8, 0.0, 0.5),
    ActionCommand(HighLevelAction.ROTATE_LEFT, 0.5, 0.0, 0.3),
    ActionCommand(HighLevelAction.GESTURE_WAVE, 1.0, 0.0, 0.8),
]

print("Example high-level actions:\n")
for action in example_actions:
    print(f"  {action.get_description()}")
    motion_cmd = action.to_motion_command()
    print(f"    → Velocity: {motion_cmd['velocity']}")
    print(f"    → Gesture: {motion_cmd['is_gesture']}\n")

---
<a id='primitives'></a>
## 4. Motion Primitives Library

Motion primitives are **pre-validated movement sequences** that abstract low-level control.

In [ ]:
class MotionPrimitive:
    """Base class for motion primitives"""
    
    def __init__(self, name: str, duration: float):
        self.name = name
        self.duration = duration  # seconds
        self.is_executing = False
        self.start_time = None
    
    def execute(self, robot_interface) -> bool:
        """
        Execute the motion primitive
        
        Args:
            robot_interface: Connection to real or simulated robot
        
        Returns:
            completed: True if primitive finished
        """
        raise NotImplementedError
    
    def reset(self):
        """Reset primitive state"""
        self.is_executing = False
        self.start_time = None

class LocomotionPrimitive(MotionPrimitive):
    """Locomotion primitive: walk with specified velocity"""
    
    def __init__(self, name: str, target_velocity: np.ndarray, duration: float = 1.0):
        super().__init__(name, duration)
        self.target_velocity = target_velocity  # [vx, vy, vw]
    
    def execute(self, robot_interface) -> bool:
        if not self.is_executing:
            self.is_executing = True
            self.start_time = time.time()
        
        elapsed = time.time() - self.start_time
        
        if elapsed < self.duration:
            # Send velocity command
            if hasattr(robot_interface, 'send_velocity'):
                robot_interface.send_velocity(self.target_velocity)
            return False  # Still executing
        else:
            # Completed
            self.reset()
            return True

class GesturePrimitive(MotionPrimitive):
    """Gesture primitive: pre-defined arm motion"""
    
    def __init__(self, name: str, gesture_id: int, duration: float = 3.0):
        super().__init__(name, duration)
        self.gesture_id = gesture_id
    
    def execute(self, robot_interface) -> bool:
        if not self.is_executing:
            self.is_executing = True
            self.start_time = time.time()
            
            # Trigger gesture
            if hasattr(robot_interface, 'execute_gesture'):
                robot_interface.execute_gesture(self.gesture_id)
        
        elapsed = time.time() - self.start_time
        
        if elapsed >= self.duration:
            self.reset()
            return True
        return False

class AudioPrimitive(MotionPrimitive):
    """Audio primitive: play sound or TTS"""
    
    def __init__(self, name: str, message: str, volume: float = 0.5):
        super().__init__(name, duration=2.0)
        self.message = message
        self.volume = volume
    
    def execute(self, robot_interface) -> bool:
        if not self.is_executing:
            self.is_executing = True
            self.start_time = time.time()
            
            # Play audio
            if hasattr(robot_interface, 'speak'):
                robot_interface.speak(self.message, self.volume)
        
        elapsed = time.time() - self.start_time
        
        if elapsed >= self.duration:
            self.reset()
            return True
        return False

class MotionPrimitiveLibrary:
    """Library of pre-defined motion primitives"""
    
    def __init__(self):
        self.primitives = {}
        self._initialize_primitives()
    
    def _initialize_primitives(self):
        """Create standard motion primitives"""
        # Locomotion primitives
        self.primitives['walk_forward'] = LocomotionPrimitive(
            'walk_forward', np.array([0.3, 0, 0]), duration=1.0
        )
        self.primitives['walk_backward'] = LocomotionPrimitive(
            'walk_backward', np.array([-0.2, 0, 0]), duration=1.0
        )
        self.primitives['strafe_left'] = LocomotionPrimitive(
            'strafe_left', np.array([0, 0.2, 0]), duration=1.0
        )
        self.primitives['strafe_right'] = LocomotionPrimitive(
            'strafe_right', np.array([0, -0.2, 0]), duration=1.0
        )
        self.primitives['rotate_left'] = LocomotionPrimitive(
            'rotate_left', np.array([0, 0, 0.5]), duration=1.0
        )
        self.primitives['rotate_right'] = LocomotionPrimitive(
            'rotate_right', np.array([0, 0, -0.5]), duration=1.0
        )
        
        # Gesture primitives
        self.primitives['wave'] = GesturePrimitive('wave', gesture_id=0, duration=3.0)
        self.primitives['handshake'] = GesturePrimitive('handshake', gesture_id=1, duration=4.0)
        self.primitives['clap'] = GesturePrimitive('clap', gesture_id=2, duration=3.0)
        
        # Audio primitives
        self.primitives['greet'] = AudioPrimitive('greet', "Hello, nice to meet you!")
        self.primitives['acknowledge'] = AudioPrimitive('acknowledge', "Understood")
    
    def get_primitive(self, name: str) -> Optional[MotionPrimitive]:
        """Retrieve primitive by name"""
        return self.primitives.get(name)
    
    def list_primitives(self) -> List[str]:
        """List all available primitives"""
        return list(self.primitives.keys())

# Initialize library
motion_lib = MotionPrimitiveLibrary()
print("Motion Primitive Library initialized\n")
print("Available primitives:")
for name in motion_lib.list_primitives():
    primitive = motion_lib.get_primitive(name)
    print(f"  - {name:20s} (duration: {primitive.duration:.1f}s)")

---
<a id='rewards'></a>
## 5. Reward Function Design

A well-designed reward function is critical for RL success. We'll use a **simple, sparse reward** approach.

### Reward Components

#### 1. Goal Reaching Reward (+100)
- Large positive reward when robot reaches goal
- Encourages task completion

#### 2. Progress Reward (+0.1 per step)
- Small reward for moving closer to goal
- Provides learning signal before goal is reached

#### 3. Command Following Reward (+0.5)
- Reward for following human controller commands
- Encourages responsive behavior

#### 4. Stability Penalty (-1.0 per fall)
- Negative reward if robot falls (large tilt)
- Ensures safety

#### 5. Collision Penalty (-0.5)
- Penalty for hitting obstacles
- Encourages obstacle avoidance

#### 6. Energy Penalty (-0.001 per action)
- Small penalty for movement
- Encourages efficiency

**Total Reward:**
```
r = r_goal + r_progress + r_command + r_stability + r_collision + r_energy
```

In [ ]:
class RewardFunction:
    """Compute reward for high-level RL"""
    
    def __init__(self):
        # Reward weights
        self.w_goal = 100.0
        self.w_progress = 0.1
        self.w_command = 0.5
        self.w_stability = -1.0
        self.w_collision = -0.5
        self.w_energy = -0.001
        
        # Thresholds
        self.goal_radius = 0.5  # meters
        self.fall_angle = 0.5  # radians (~30 degrees)
        self.collision_distance = 0.3  # meters
        
        # Tracking
        self.previous_goal_distance = None
    
    def compute_reward(
        self,
        state: RobotState,
        action: ActionCommand,
        next_state: RobotState
    ) -> Tuple[float, Dict[str, float]]:
        """
        Compute reward for state-action-next_state transition
        
        Returns:
            total_reward: Scalar reward
            reward_breakdown: Dictionary with individual components
        """
        rewards = {}
        
        # 1. Goal reaching
        reached_goal = next_state.goal_distance < self.goal_radius
        rewards['goal'] = self.w_goal if reached_goal else 0.0
        
        # 2. Progress toward goal
        if self.previous_goal_distance is not None:
            progress = self.previous_goal_distance - next_state.goal_distance
            rewards['progress'] = self.w_progress * progress
        else:
            rewards['progress'] = 0.0
        
        self.previous_goal_distance = next_state.goal_distance
        
        # 3. Command following (alignment with human commands)
        # Measure how well action aligns with commanded velocity
        motion_cmd = action.to_motion_command()
        commanded_vel = state.cmd_velocity
        actual_vel = motion_cmd['velocity']
        
        # Cosine similarity (if non-zero commands)
        if np.linalg.norm(commanded_vel) > 0.01:
            alignment = np.dot(commanded_vel, actual_vel) / (
                np.linalg.norm(commanded_vel) * (np.linalg.norm(actual_vel) + 1e-6)
            )
            rewards['command'] = self.w_command * max(0, alignment)
        else:
            rewards['command'] = 0.0
        
        # 4. Stability penalty
        roll, pitch, _ = next_state.base_orientation
        fell = abs(roll) > self.fall_angle or abs(pitch) > self.fall_angle
        rewards['stability'] = self.w_stability if fell else 0.0
        
        # 5. Collision penalty
        min_obstacle_dist = np.min(next_state.obstacle_proximity)
        collided = min_obstacle_dist < self.collision_distance
        rewards['collision'] = self.w_collision if collided else 0.0
        
        # 6. Energy penalty (encourage efficiency)
        action_magnitude = np.linalg.norm(motion_cmd['velocity'])
        rewards['energy'] = self.w_energy * action_magnitude
        
        # Total reward
        total_reward = sum(rewards.values())
        
        return total_reward, rewards
    
    def reset(self):
        """Reset tracking variables"""
        self.previous_goal_distance = None

# Example reward computation
reward_fn = RewardFunction()

# Simulate scenario: robot moving toward goal
state = RobotState(
    base_orientation=np.array([0.05, 0.02, 0.0]),
    base_angular_velocity=np.zeros(3),
    base_linear_velocity=np.array([0.3, 0, 0]),
    joint_positions=np.zeros(29),
    joint_velocities=np.zeros(29),
    foot_contacts=np.array([1, 1]),
    goal_direction=np.array([1, 0, 0]),
    goal_distance=5.0,
    obstacle_proximity=np.ones(8) * 10.0,
    cmd_velocity=np.array([0.3, 0, 0]),
    cmd_mode=1,
    cmd_gesture=0
)

action = ActionCommand(HighLevelAction.WALK_FORWARD, 0.8, 0.0, 0.5)

next_state = RobotState(
    base_orientation=np.array([0.03, 0.01, 0.0]),
    base_angular_velocity=np.zeros(3),
    base_linear_velocity=np.array([0.3, 0, 0]),
    joint_positions=np.zeros(29),
    joint_velocities=np.zeros(29),
    foot_contacts=np.array([1, 1]),
    goal_direction=np.array([1, 0, 0]),
    goal_distance=4.7,  # Moved 0.3m closer
    obstacle_proximity=np.ones(8) * 10.0,
    cmd_velocity=np.array([0.3, 0, 0]),
    cmd_mode=1,
    cmd_gesture=0
)

reward, breakdown = reward_fn.compute_reward(state, action, next_state)

print("Reward Computation Example:\n")
print(f"Action: {action.get_description()}")
print(f"\nReward Breakdown:")
for component, value in breakdown.items():
    print(f"  {component:15s}: {value:+7.4f}")
print(f"\nTotal Reward: {reward:+7.4f}")

---
<a id='environment'></a>
## 6. Gymnasium Environment Implementation

We'll implement a Gymnasium environment that wraps everything together.

In [ ]:
class G1HighLevelEnv(gym.Env):
    """
    High-level RL environment for Unitree G1
    
    State: 86D (robot obs + controller commands)
    Action: Discrete(10) + Box(3) (high-level actions + parameters)
    """
    
    def __init__(self, use_controller: bool = True, simulation: bool = True):
        super().__init__()
        
        self.use_controller = use_controller
        self.simulation = simulation
        
        # Initialize components
        self.controller = USBControllerInput() if use_controller else None
        self.motion_lib = MotionPrimitiveLibrary()
        self.reward_fn = RewardFunction()
        
        # Define spaces
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(86,),
            dtype=np.float32
        )
        
        # Action space: discrete action + continuous parameters
        self.action_space = spaces.Dict({
            'action_type': spaces.Discrete(10),
            'parameters': spaces.Box(
                low=np.array([0.0, -1.0, 0.0]),
                high=np.array([1.0, 1.0, 1.0]),
                dtype=np.float32
            )
        })
        
        # Episode parameters
        self.max_steps = 1000
        self.current_step = 0
        
        # Robot state (initialized in reset)
        self.state = None
        self.goal_position = np.array([5.0, 0.0, 0.0])  # 5m ahead
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # Reset step counter
        self.current_step = 0
        
        # Reset reward function
        self.reward_fn.reset()
        
        # Initialize robot state
        self.state = RobotState(
            base_orientation=np.zeros(3),
            base_angular_velocity=np.zeros(3),
            base_linear_velocity=np.zeros(3),
            joint_positions=np.zeros(29),
            joint_velocities=np.zeros(29),
            foot_contacts=np.array([1, 1]),
            goal_direction=self.goal_position / np.linalg.norm(self.goal_position),
            goal_distance=np.linalg.norm(self.goal_position),
            obstacle_proximity=np.ones(8) * 10.0,
            cmd_velocity=np.zeros(3),
            cmd_mode=1,
            cmd_gesture=0
        )
        
        # Read controller if available
        if self.controller and self.controller.available:
            commands = self.controller.read_commands()
            self.state.cmd_velocity = commands['velocity']
            self.state.cmd_mode = commands['mode']
            self.state.cmd_gesture = commands['gesture']
        
        return self.state.to_array(), {}
    
    def step(self, action):
        # Parse action
        action_type = HighLevelAction(action['action_type'])
        velocity_scale, direction_bias, audio_volume = action['parameters']
        
        action_cmd = ActionCommand(
            action_type=action_type,
            velocity_scale=float(velocity_scale),
            direction_bias=float(direction_bias),
            audio_volume=float(audio_volume)
        )
        
        # Execute action (simulate robot dynamics)
        next_state = self._simulate_step(action_cmd)
        
        # Read new controller commands
        if self.controller and self.controller.available:
            commands = self.controller.read_commands()
            next_state.cmd_velocity = commands['velocity']
            next_state.cmd_mode = commands['mode']
            next_state.cmd_gesture = commands['gesture']
        
        # Compute reward
        reward, reward_breakdown = self.reward_fn.compute_reward(
            self.state, action_cmd, next_state
        )
        
        # Check termination
        self.current_step += 1
        
        goal_reached = next_state.goal_distance < self.reward_fn.goal_radius
        fell = abs(next_state.base_orientation[0]) > self.reward_fn.fall_angle or \
               abs(next_state.base_orientation[1]) > self.reward_fn.fall_angle
        
        terminated = goal_reached or fell
        truncated = self.current_step >= self.max_steps
        
        # Update state
        self.state = next_state
        
        info = {
            'goal_reached': goal_reached,
            'fell': fell,
            'goal_distance': next_state.goal_distance,
            'reward_breakdown': reward_breakdown
        }
        
        return next_state.to_array(), reward, terminated, truncated, info
    
    def _simulate_step(self, action: ActionCommand) -> RobotState:
        """
        Simulate robot dynamics for one timestep
        
        In real deployment, this would be replaced with actual robot interface
        """
        dt = 0.1  # 10 Hz control
        
        # Get motion command
        motion_cmd = action.to_motion_command()
        velocity = motion_cmd['velocity']
        
        # Simple kinematic update (placeholder physics)
        current_pos = self.goal_position - self.state.goal_direction * self.state.goal_distance
        
        # Update position based on velocity
        new_pos = current_pos + velocity[:2] * dt  # Only x, y (2D navigation)
        
        # Update goal distance
        new_goal_vector = self.goal_position[:2] - new_pos
        new_goal_distance = np.linalg.norm(new_goal_vector)
        new_goal_direction = np.append(
            new_goal_vector / (new_goal_distance + 1e-6), [0]
        )  # Normalize + add z=0
        
        # Add small noise to orientation (simulate walking dynamics)
        orientation_noise = np.random.normal(0, 0.02, size=3)
        new_orientation = self.state.base_orientation + orientation_noise
        
        # Create next state
        next_state = RobotState(
            base_orientation=new_orientation,
            base_angular_velocity=self.state.base_angular_velocity * 0.9,  # Damping
            base_linear_velocity=velocity,
            joint_positions=self.state.joint_positions,  # Simplified
            joint_velocities=self.state.joint_velocities * 0.9,
            foot_contacts=self.state.foot_contacts,
            goal_direction=new_goal_direction,
            goal_distance=new_goal_distance,
            obstacle_proximity=self.state.obstacle_proximity,
            cmd_velocity=self.state.cmd_velocity,
            cmd_mode=self.state.cmd_mode,
            cmd_gesture=self.state.cmd_gesture
        )
        
        return next_state
    
    def render(self):
        """Optional rendering (not implemented)"""
        pass
    
    def close(self):
        """Cleanup"""
        if self.controller and PYGAME_AVAILABLE:
            pygame.quit()

# Test environment
print("Creating high-level RL environment...")
env = G1HighLevelEnv(use_controller=False, simulation=True)

print(f"\nObservation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

# Test episode
obs, info = env.reset()
print(f"\nInitial observation shape: {obs.shape}")

for i in range(5):
    action = {
        'action_type': 1,  # WALK_FORWARD
        'parameters': np.array([0.8, 0.0, 0.5])
    }
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {i+1}: reward={reward:.4f}, goal_dist={info['goal_distance']:.2f}m")
    
    if terminated or truncated:
        break

print("\n✓ Environment test complete")

---
<a id='training'></a>
## 7. Training with PPO

Train the policy using Proximal Policy Optimization (PPO).

In [ ]:
# Custom wrapper to flatten Dict action space for SB3
class FlattenedActionWrapper(gym.Wrapper):
    """Flatten Dict action space to Box for compatibility"""
    
    def __init__(self, env):
        super().__init__(env)
        
        # Flattened action: [action_type (1), parameters (3)] = 4D
        self.action_space = spaces.Box(
            low=np.array([0.0, 0.0, -1.0, 0.0]),
            high=np.array([9.0, 1.0, 1.0, 1.0]),
            dtype=np.float32
        )
    
    def step(self, action):
        # Convert flattened action back to dict
        action_type = int(np.clip(action[0], 0, 9))
        parameters = action[1:]
        
        dict_action = {
            'action_type': action_type,
            'parameters': parameters
        }
        
        return self.env.step(dict_action)

# Training callback to log progress
class TrainingCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.episode_rewards = []
        self.episode_lengths = []
        self.current_episode_reward = 0
        self.current_episode_length = 0
    
    def _on_step(self) -> bool:
        self.current_episode_reward += self.locals['rewards'][0]
        self.current_episode_length += 1
        
        if self.locals['dones'][0]:
            self.episode_rewards.append(self.current_episode_reward)
            self.episode_lengths.append(self.current_episode_length)
            
            if len(self.episode_rewards) % 10 == 0:
                avg_reward = np.mean(self.episode_rewards[-10:])
                avg_length = np.mean(self.episode_lengths[-10:])
                print(f"Episode {len(self.episode_rewards)}: "
                      f"avg_reward={avg_reward:.2f}, avg_length={avg_length:.1f}")
            
            self.current_episode_reward = 0
            self.current_episode_length = 0
        
        return True

if SB3_AVAILABLE:
    print("🎓 Starting PPO training...\n")
    
    # Create environment with wrapper
    train_env = FlattenedActionWrapper(G1HighLevelEnv(use_controller=False, simulation=True))
    
    # Create PPO model
    model = PPO(
        policy="MlpPolicy",
        env=train_env,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        verbose=1,
        device='cpu'  # Change to 'cuda' if GPU available
    )
    
    # Train
    callback = TrainingCallback()
    
    print("Training for 50,000 timesteps (increase for better performance)...\n")
    model.learn(
        total_timesteps=50000,
        callback=callback,
        progress_bar=True
    )
    
    # Save model
    model.save("g1_high_level_ppo")
    print("\n✓ Model saved to g1_high_level_ppo.zip")
    
    # Plot training progress
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Episode rewards
    ax1.plot(callback.episode_rewards, alpha=0.3, label='Episode Reward')
    window = 10
    if len(callback.episode_rewards) > window:
        moving_avg = np.convolve(
            callback.episode_rewards, 
            np.ones(window)/window, 
            mode='valid'
        )
        ax1.plot(range(window-1, len(callback.episode_rewards)), moving_avg, 
                'r-', linewidth=2, label=f'{window}-Episode Moving Average')
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Total Reward')
    ax1.set_title('Training Progress: Rewards')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Episode lengths
    ax2.plot(callback.episode_lengths, alpha=0.3, label='Episode Length')
    if len(callback.episode_lengths) > window:
        moving_avg = np.convolve(
            callback.episode_lengths, 
            np.ones(window)/window, 
            mode='valid'
        )
        ax2.plot(range(window-1, len(callback.episode_lengths)), moving_avg, 
                'g-', linewidth=2, label=f'{window}-Episode Moving Average')
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Steps')
    ax2.set_title('Training Progress: Episode Length')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_progress.png', dpi=150)
    plt.show()
    
    print("\n📊 Training visualization saved to training_progress.png")
    
else:
    print("⚠️  Stable-Baselines3 not available - skipping training")

---
<a id='deployment'></a>
## 8. Deployment to Real Robot

Deploy the trained policy to the physical Unitree G1.

In [ ]:
class G1RobotInterface:
    """Interface to real Unitree G1 hardware"""
    
    def __init__(self, network_interface: str = "eth0"):
        self.network_interface = network_interface
        self.initialized = False
        
        if UNITREE_SDK_AVAILABLE:
            try:
                # Initialize DDS
                ChannelFactoryInitialize(0, network_interface)
                
                # Create clients
                self.loco_client = LocoClient()
                self.loco_client.SetTimeout(10.0)
                self.loco_client.Init()
                
                self.audio_client = AudioClient()
                self.audio_client.SetTimeout(10.0)
                self.audio_client.Init()
                
                self.arm_client = G1ArmActionClient()
                self.arm_client.SetTimeout(10.0)
                self.arm_client.Init()
                
                # Subscribe to state
                self.state_subscriber = ChannelSubscriber("rt/lowstate", LowState_)
                self.state_subscriber.Init(self._state_callback, 10)
                
                self.current_state = None
                self.initialized = True
                
                print(f"✓ Connected to G1 on {network_interface}")
                
            except Exception as e:
                print(f"⚠️  Failed to connect to robot: {e}")
        else:
            print("⚠️  Unitree SDK not available")
    
    def _state_callback(self, msg: LowState_):
        """Store latest robot state"""
        self.current_state = msg
    
    def get_state(self) -> Optional[RobotState]:
        """
        Read current robot state
        
        Returns:
            state: RobotState object or None if unavailable
        """
        if not self.initialized or self.current_state is None:
            return None
        
        # Extract data from low-level state
        imu = self.current_state.imu_state
        motors = self.current_state.motor_state
        
        # Build RobotState
        state = RobotState(
            base_orientation=np.array(imu.rpy),
            base_angular_velocity=np.array(imu.gyroscope),
            base_linear_velocity=np.array([0, 0, 0]),  # Not directly available
            joint_positions=np.array([m.q for m in motors]),
            joint_velocities=np.array([m.dq for m in motors]),
            foot_contacts=np.array([1, 1]),  # Placeholder (need force sensors)
            goal_direction=np.array([1, 0, 0]),  # Set externally
            goal_distance=5.0,  # Set externally
            obstacle_proximity=np.ones(8) * 10.0,  # From depth camera
            cmd_velocity=np.zeros(3),  # From controller
            cmd_mode=1,
            cmd_gesture=0
        )
        
        return state
    
    def send_velocity(self, velocity: np.ndarray):
        """Send velocity command to robot"""
        if not self.initialized:
            return
        
        vx, vy, vw = velocity
        self.loco_client.Move(float(vx), float(vy), float(vw))
    
    def execute_gesture(self, gesture_id: int):
        """Execute pre-defined gesture"""
        if not self.initialized:
            return
        
        gesture_map = {
            0: "high wave",
            1: "shake hand",
            2: "clap"
        }
        
        gesture_name = gesture_map.get(gesture_id)
        if gesture_name and gesture_name in action_map:
            self.arm_client.ExecuteAction(action_map[gesture_name])
            time.sleep(0.1)
            self.arm_client.ExecuteAction(action_map["release arm"])
    
    def speak(self, message: str, volume: float = 0.5):
        """Text-to-speech output"""
        if not self.initialized:
            return
        
        # Set volume (0-100)
        self.audio_client.SetVolume(int(volume * 100))
        
        # Chinese TTS (G1 default)
        self.audio_client.TtsMaker(message, 0)
    
    def emergency_stop(self):
        """Emergency stop"""
        if not self.initialized:
            return
        
        self.loco_client.Damp()
        print("⚠️  EMERGENCY STOP ACTIVATED")

class DeployedPolicy:
    """Deployed RL policy running on real robot"""
    
    def __init__(self, model_path: str, network_interface: str = "eth0"):
        # Load trained model
        if SB3_AVAILABLE:
            self.model = PPO.load(model_path)
            print(f"✓ Loaded policy from {model_path}")
        else:
            self.model = None
            print("⚠️  SB3 not available")
        
        # Connect to robot
        self.robot = G1RobotInterface(network_interface)
        
        # Controller for human input
        self.controller = USBControllerInput()
        
        # Goal tracking
        self.goal_position = np.array([5.0, 0.0, 0.0])
    
    def run(self, duration: float = 60.0):
        """
        Run policy on real robot
        
        Args:
            duration: How long to run (seconds)
        """
        if not self.robot.initialized or self.model is None:
            print("⚠️  Cannot run: robot or model not initialized")
            return
        
        print(f"🤖 Running policy for {duration:.0f} seconds...")
        print("   Press START button on controller for emergency stop\n")
        
        # Stand up
        self.robot.loco_client.Squat2StandUp()
        time.sleep(2)
        
        start_time = time.time()
        step_count = 0
        
        while time.time() - start_time < duration:
            # Get robot state
            state = self.robot.get_state()
            if state is None:
                time.sleep(0.1)
                continue
            
            # Get controller commands
            if self.controller.available:
                commands = self.controller.read_commands()
                state.cmd_velocity = commands['velocity']
                state.cmd_mode = commands['mode']
                state.cmd_gesture = commands['gesture']
                
                # Check emergency stop
                if commands['emergency_stop']:
                    self.robot.emergency_stop()
                    break
            
            # Update goal tracking (would come from navigation system)
            current_pos = np.array([0, 0, 0])  # Placeholder
            goal_vec = self.goal_position - current_pos
            state.goal_distance = np.linalg.norm(goal_vec)
            state.goal_direction = goal_vec / (state.goal_distance + 1e-6)
            
            # Get action from policy
            obs = state.to_array()
            action_flat, _ = self.model.predict(obs, deterministic=True)
            
            # Parse action
            action_type = HighLevelAction(int(np.clip(action_flat[0], 0, 9)))
            velocity_scale, direction_bias, audio_volume = action_flat[1:]
            
            action_cmd = ActionCommand(
                action_type=action_type,
                velocity_scale=float(velocity_scale),
                direction_bias=float(direction_bias),
                audio_volume=float(audio_volume)
            )
            
            # Execute action
            motion_cmd = action_cmd.to_motion_command()
            
            if motion_cmd['is_gesture']:
                self.robot.execute_gesture(motion_cmd['gesture_id'])
            else:
                self.robot.send_velocity(motion_cmd['velocity'])
            
            # Log progress
            if step_count % 10 == 0:
                print(f"\rStep {step_count}: {action_cmd.get_description()}", 
                      end='', flush=True)
            
            step_count += 1
            time.sleep(0.1)  # 10 Hz control loop
        
        # Stop robot
        self.robot.loco_client.Move(0, 0, 0)
        print("\n\n✓ Policy execution complete")

# Example deployment (requires real robot)
print("\nDeployment Example:\n")
print("To deploy trained policy to real G1:")
print("""\n```python
# Load and run policy
policy = DeployedPolicy(
    model_path='g1_high_level_ppo.zip',
    network_interface='eth0'
)

# Run for 60 seconds
policy.run(duration=60.0)
```""")
print("\n⚠️  Ensure robot is in safe environment before running!")

---
<a id='applications'></a>
## 9. Example Applications

### Application 1: Interactive Tour Guide

Robot follows human operator and responds to gesture commands.

In [ ]:
class TourGuideApplication:
    """
    Tour guide robot that follows human and performs gestures
    """
    
    def __init__(self, policy_path: str):
        self.deployed_policy = DeployedPolicy(policy_path)
        self.waypoints = [
            ("Entrance", np.array([0, 0, 0])),
            ("Exhibit A", np.array([5, 0, 0])),
            ("Exhibit B", np.array([5, 5, 0])),
            ("Exit", np.array([0, 5, 0]))
        ]
        self.current_waypoint = 0
    
    def start_tour(self):
        """
        Execute guided tour with narration
        """
        print("🎭 Starting tour guide mode...")
        
        for name, position in self.waypoints:
            print(f"\nNavigating to: {name}")
            
            # Update goal
            self.deployed_policy.goal_position = position
            
            # Wave when arriving
            self.deployed_policy.robot.execute_gesture(0)  # Wave
            
            # Narration (would be location-specific)
            self.deployed_policy.robot.speak(f"Welcome to {name}")
            
            # Wait at location
            time.sleep(5)
        
        print("\n✓ Tour complete!")

print("\n📝 Example: Tour Guide Application")
print("   - Robot follows pre-defined waypoints")
print("   - Waves and provides audio narration at each location")
print("   - Responds to human controller for manual override")

### Application 2: Warehouse Assistant

Robot navigates warehouse and performs pick-and-place tasks.

In [ ]:
class WarehouseAssistant:
    """
    Warehouse robot for logistics tasks
    """
    
    def __init__(self, policy_path: str):
        self.deployed_policy = DeployedPolicy(policy_path)
        self.inventory = {
            'A1': 'Computer Monitor',
            'A2': 'Keyboard',
            'B1': 'Mouse',
            'B2': 'Headphones'
        }
    
    def pick_item(self, location: str) -> bool:
        """
        Navigate to location and pick item
        
        Returns:
            success: True if item retrieved
        """
        item_name = self.inventory.get(location, "Unknown")
        print(f"\n📦 Retrieving: {item_name} from {location}")
        
        # Navigate to location (would use real coordinates)
        # For demo, use placeholder
        self.deployed_policy.goal_position = np.array([5, 0, 0])
        
        # Audio feedback
        self.deployed_policy.robot.speak(f"Retrieving {item_name}")
        
        # Simulate pick (would use manipulation skills)
        time.sleep(3)
        
        print(f"✓ Item retrieved: {item_name}")
        return True
    
    def deliver_item(self, destination: str) -> bool:
        """
        Deliver held item to destination
        """
        print(f"\n🚚 Delivering to: {destination}")
        
        # Navigate to destination
        # ...
        
        # Place item
        self.deployed_policy.robot.speak("Delivery complete")
        
        return True

print("\n📝 Example: Warehouse Assistant")
print("   - Autonomous navigation to storage locations")
print("   - Audio confirmation of tasks")
print("   - Human operator can override via controller")

---
## Summary & Next Steps

This notebook demonstrated a **complete high-level RL framework** for the Unitree G1:

✅ **State Space:** Combined robot sensors + human controller input (86D)  
✅ **Action Space:** Abstract high-level actions (locomotion, gestures, audio)  
✅ **Motion Primitives:** Pre-validated movement sequences  
✅ **Reward Function:** Simple sparse rewards for goal reaching and command following  
✅ **Training:** PPO implementation with Stable-Baselines3  
✅ **Deployment:** Real robot interface with safety features  
✅ **Applications:** Tour guide and warehouse assistant examples

### Key Advantages of High-Level RL

1. **Faster Learning:** Smaller action space than low-level joint control
2. **Better Generalization:** Abstractions transfer across tasks
3. **Safer:** Primitives are pre-tested and bounded
4. **Human-Interpretable:** Actions have semantic meaning
5. **Controller Integration:** Human can guide learning and provide real-time input

### Next Steps

1. **Expand Motion Library:** Add more primitives (stairs, doors, obstacles)
2. **Vision Integration:** Add camera observations to state space
3. **Multi-Task Learning:** Train single policy for multiple applications
4. **Hierarchical RL:** Add meta-controller for task planning
5. **Human Preference Learning:** Fine-tune policy using human feedback
6. **🆕 LLM-Based Agentic Control:** High-level task planning with language models (see next section!)

### References

- **Hierarchical RL:** "Options Framework" (Sutton, Precup, Singh)
- **Human-in-the-Loop:** "Deep TAMER" (Warnell et al.)
- **PPO Algorithm:** "Proximal Policy Optimization" (Schulman et al.)
- **Motion Primitives:** "Dynamic Movement Primitives" (Schaal et al.)

---

*High-Level RL Framework for Unitree G1*  
*Enables rapid task learning with human guidance*

---
## LLM-Based Control: Summary & Advanced Topics

### What We Built

This section implemented a **complete LLM-based agentic control system** that combines:

✅ **Natural Language Interface:** Control robot with plain English commands  
✅ **RAG Knowledge Base:** Context from manuals, code repos, API docs, and past executions  
✅ **LLM Agent Planner:** HuggingFace models for intelligent action planning  
✅ **RLHF Reward Model:** Secondary LLM evaluates and scores performance  
✅ **Integrated Execution:** Seamless connection to existing RL policies  
✅ **Continuous Learning:** Feedback collection for model fine-tuning  

### System Architecture

```
┌──────────────────────────────────────────────────────────────┐
│                    User Command (NL)                         │
│                 "Walk to door and wave"                      │
└────────────────────┬─────────────────────────────────────────┘
                     │
                     ▼
┌──────────────────────────────────────────────────────────────┐
│              RAG System Retrieval                            │
│  ┌────────────┐  ┌────────────┐  ┌─────────────┐           │
│  │  Manuals   │  │   GitHub   │  │  Past Logs  │           │
│  └────────────┘  └────────────┘  └─────────────┘           │
│         Relevant context for planning (vector search)        │
└────────────────────┬─────────────────────────────────────────┘
                     │
                     ▼
┌──────────────────────────────────────────────────────────────┐
│          LLM Agent Controller (Planning)                     │
│  Model: Llama-3.1-70B / Mixtral-8x7B / Phi-3               │
│  Output: ["walk_forward", "idle", "wave"]                   │
│          + reasoning + safety validation                     │
└────────────────────┬─────────────────────────────────────────┘
                     │
                     ▼
┌──────────────────────────────────────────────────────────────┐
│         Motion Primitive Execution                           │
│  Uses existing RL policy + Unitree SDK                      │
│  Monitors state, detects failures                           │
└────────────────────┬─────────────────────────────────────────┘
                     │
                     ▼
┌──────────────────────────────────────────────────────────────┐
│          RLHF Reward Model (Evaluation)                      │
│  Scores: Safety, Efficiency, Goal Progress, Naturalness    │
│  Provides: Reward signal + textual feedback                 │
└────────────────────┬─────────────────────────────────────────┘
                     │
                     ▼
┌──────────────────────────────────────────────────────────────┐
│         Feedback Loop (Learning)                             │
│  Stores execution logs in RAG for future queries           │
│  Exports training data for LLM fine-tuning                  │
└──────────────────────────────────────────────────────────────┘
```

### Key Advantages Over Pure RL

| Aspect | Traditional RL | LLM-Based Control |
|--------|---------------|-------------------|
| **Interface** | Numeric state/actions | Natural language |
| **Learning Speed** | Requires 100K+ episodes | Few-shot from documentation |
| **Generalization** | Task-specific | Zero-shot to new tasks |
| **Explainability** | Black box policy | Reasoning provided |
| **Human Feedback** | Requires reward engineering | Direct evaluation via RLHF |
| **Knowledge Integration** | Limited to training data | RAG from unlimited sources |

### Setup Instructions

#### 1. Get HuggingFace API Token

```python
# Get free token from: https://huggingface.co/settings/tokens
import os
os.environ["HUGGINGFACE_TOKEN"] = "hf_..."  # Your token here
```

#### 2. Choose LLM Model

Available models (via HuggingFace Inference API):

- **Llama-3.1-70B-Instruct** - Best quality, slower (recommended)
- **Mixtral-8x7B-Instruct-v0.1** - Fast, good balance
- **Phi-3-medium-4k-instruct** - Lightweight, fastest
- **Qwen/Qwen2.5-72B-Instruct** - Strong multilingual support

```python
llm_controller = LLMAgentController(
    model_name="meta-llama/Llama-3.1-70B-Instruct",
    knowledge_base=knowledge_base,
    use_rag=True
)
```

#### 3. Populate Knowledge Base

```python
# Add your robot documentation
knowledge_base.add_manual_documentation(
    text=open("G1_Manual.pdf").read(),
    source="Official_Manual"
)

# Add code from GitHub
knowledge_base.add_github_code(
    code=open("examples/navigation.py").read(),
    repo="unitree/unitree_sdk2",
    file_path="examples/navigation.py"
)

# Add API docs
knowledge_base.add_api_documentation({
    "function_name": "description..."
})
```

#### 4. Deploy to Real Robot

```python
# Connect to real G1
robot = G1RobotInterface(network_interface="eth0")

# Create controller
controller = LLMBasedRobotController(
    knowledge_base=knowledge_base,
    llm_agent=llm_controller,
    rlhf_model=rlhf_reward_model,
    robot_interface=robot  # Use real robot instead of sim
)

# Execute commands
controller.execute_command("Walk to the meeting room and greet visitors")
```

### Advanced Topics

#### Fine-Tuning the LLM Agent

After collecting execution data, fine-tune for robot-specific control:

```python
# Export training data
controller.export_training_data("robot_training.json")

# Use HuggingFace AutoTrain or custom training
# Format: instruction → actions + reasoning pairs
# Example entry:
{
    "instruction": "Walk forward and wave",
    "output": {
        "actions": ["walk_forward", "idle", "wave"],
        "reasoning": "Break into locomotion then gesture for safety"
    }
}
```

#### Training a Custom Reward Model

Fine-tune a smaller model on collected feedback:

```python
# Export RLHF feedback
rlhf_model.export_feedback_dataset("rlhf_training.json")

# Train reward model to predict scores
# Input: (command, actions, state_delta)
# Output: (safety, efficiency, progress, naturalness scores)
```

#### Multimodal Integration

Add vision for enhanced understanding:

```python
# Add camera observations to state
state.camera_image = robot.get_camera_image()

# Use multimodal LLM (e.g., LLaVA, GPT-4V)
# Provides visual grounding for commands like:
# "Pick up the red box on the table"
```

#### Hierarchical LLM Control

Use multiple LLMs at different abstraction levels:

```python
# High-level task planner
task_plan = meta_llm.plan("Clean the office")
# → ["go_to_desk", "pick_trash", "go_to_bin", "dispose"]

# Low-level action planner (our LLM controller)
for task in task_plan:
    actions = llm_controller.plan_actions(task)
    execute(actions)
```

### Limitations & Future Work

**Current Limitations:**
- LLM latency (1-3 seconds per query)
- Requires internet connection for HF API
- Token costs for extensive use
- Limited to predefined action primitives

**Future Improvements:**
1. **Local LLM Deployment:** Use quantized models (GGUF format) with llama.cpp
2. **Action Space Learning:** Let LLM discover new primitives
3. **Multi-Robot Coordination:** Extend to robot teams
4. **Real-Time Adaptation:** Online learning from execution feedback
5. **Safety Guarantees:** Formal verification of LLM plans

### References & Resources

**LLM for Robotics:**
- "Do As I Can, Not As I Say" (Google, 2022) - SayCan paper
- "Code as Policies" (Google, 2023) - LLMs generate robot code
- "RT-2: Vision-Language-Action Models" (Google, 2023)
- "VoxPoser" (Stanford, 2023) - LLM spatial reasoning

**RLHF:**
- "Learning to Summarize from Human Feedback" (OpenAI, 2020)
- "Training Language Models with Human Preferences" (Anthropic, 2022)
- "Constitutional AI" (Anthropic, 2022)

**RAG Systems:**
- "Retrieval-Augmented Generation for Knowledge-Intensive Tasks" (Facebook, 2020)
- LangChain documentation: https://python.langchain.com
- ChromaDB: https://www.trychroma.com

**HuggingFace Resources:**
- Inference API: https://huggingface.co/docs/api-inference
- Model Hub: https://huggingface.co/models
- Transformers: https://huggingface.co/docs/transformers

---

*LLM-Based Agentic Control for Unitree G1*  
*Natural language interface with continuous learning from feedback*

In [ ]:
# Example commands to test the LLM-based control system
example_commands = [
    "Walk forward for a few steps",
    "Turn left and then wave hello",
    "Move backward carefully and stop",
    "Greet the person with a handshake gesture",
    "Perform a welcoming gesture"
]

print("Testing LLM-Based Robot Control System")
print("Note: Using fallback planner if HuggingFace token not configured\n")

# Execute each command
for i, cmd in enumerate(example_commands[:3]):  # Test first 3 commands
    result = llm_robot_controller.execute_command(cmd, verbose=True)
    time.sleep(0.5)  # Brief pause between commands

# Performance summary
print("\n" + "="*70)
print("Performance Summary")
print("="*70)

summary = llm_robot_controller.get_performance_summary()
print(f"\nTotal Commands: {summary.get('total_commands', 0)}")
print(f"Successful: {summary.get('successful', 0)}")
print(f"Success Rate: {summary.get('success_rate', 0)*100:.1f}%")
print(f"Average Reward: {summary.get('avg_reward', 0):+.3f}")
print(f"Average Safety Score: {summary.get('avg_safety', 0):.3f}")
print(f"Total Actions Executed: {summary.get('total_actions_executed', 0)}")

# Export training data for fine-tuning
if llm_robot_controller.execution_history:
    llm_robot_controller.export_training_data("llm_robot_training_data.json")
    rlhf_reward_model.export_feedback_dataset("rlhf_feedback_data.json")
    
    print("\n✓ Training datasets exported for fine-tuning:")
    print("  - llm_robot_training_data.json (for agent fine-tuning)")
    print("  - rlhf_feedback_data.json (for reward model fine-tuning)")

### Example: Running LLM-Based Commands

In [ ]:
class LLMBasedRobotController:
    """
    Complete LLM-based robot control system integrating:
    - Natural language understanding
    - RAG-enhanced context
    - LLM action planning
    - RL policy execution
    - RLHF reward evaluation
    """
    
    def __init__(self,
                 knowledge_base: RobotKnowledgeBase,
                 llm_agent: LLMAgentController,
                 rlhf_model: RLHFRewardModel,
                 robot_interface: Optional[G1RobotInterface] = None):
        """
        Initialize integrated control system
        
        Args:
            knowledge_base: RAG knowledge base
            llm_agent: LLM planning agent
            rlhf_model: Reward evaluation model
            robot_interface: Connection to real robot (None for simulation)
        """
        self.knowledge_base = knowledge_base
        self.llm_agent = llm_agent
        self.rlhf_model = rlhf_model
        self.robot_interface = robot_interface
        
        # Simulation environment (used when no real robot)
        self.use_simulation = robot_interface is None
        if self.use_simulation:
            self.sim_env = G1HighLevelEnv(use_controller=False, simulation=True)
            print("✓ Using simulation environment")
        else:
            print("✓ Using real robot interface")
        
        # Execution history for learning
        self.execution_history = []
        
        # Performance metrics
        self.success_count = 0
        self.total_commands = 0
    
    def execute_command(self, command: str, verbose: bool = True) -> Dict:
        """
        Execute a natural language command end-to-end
        
        Args:
            command: Natural language instruction
            verbose: Whether to print detailed logs
        
        Returns:
            result: Execution result with metrics and feedback
        """
        self.total_commands += 1
        
        if verbose:
            print("\n" + "="*70)
            print(f"💬 Command: \"{command}\"")
            print("="*70)
        
        # Step 1: Get current robot state
        if self.use_simulation:
            state_before, _ = self.sim_env.reset()
            robot_state = self.sim_env.state
        else:
            robot_state = self.robot_interface.get_state()
        
        if verbose:
            print(f"\n1️⃣  Current State:")
            print(f"   Goal distance: {robot_state.goal_distance:.2f}m")
            print(f"   Base tilt: {np.rad2deg(robot_state.base_orientation[:2])}")
        
        # Step 2: Plan actions using LLM (with RAG context)
        if verbose:
            print(f"\n2️⃣  Planning actions...")
        
        action_plan = self.llm_agent.plan_actions(command, robot_state)
        
        if not action_plan.safety_check:
            print("⚠️  SAFETY CHECK FAILED - Aborting execution")
            return {
                "success": False,
                "reason": "unsafe_plan",
                "plan": action_plan
            }
        
        # Step 3: Execute actions
        if verbose:
            print(f"\n3️⃣  Executing actions...")
        
        execution_success = True
        state_after = robot_state
        
        try:
            for i, action_name in enumerate(action_plan.actions):
                if verbose:
                    print(f"   [{i+1}/{len(action_plan.actions)}] {action_name}...", end=' ')
                
                # Map action name to HighLevelAction enum
                action_mapping = {
                    "idle": HighLevelAction.IDLE,
                    "walk_forward": HighLevelAction.WALK_FORWARD,
                    "walk_backward": HighLevelAction.WALK_BACKWARD,
                    "strafe_left": HighLevelAction.STRAFE_LEFT,
                    "strafe_right": HighLevelAction.STRAFE_RIGHT,
                    "rotate_left": HighLevelAction.ROTATE_LEFT,
                    "rotate_right": HighLevelAction.ROTATE_RIGHT,
                    "wave": HighLevelAction.GESTURE_WAVE,
                    "handshake": HighLevelAction.GESTURE_HANDSHAKE,
                    "clap": HighLevelAction.GESTURE_CLAP,
                }
                
                action_enum = action_mapping.get(action_name, HighLevelAction.IDLE)
                
                if self.use_simulation:
                    # Simulate action
                    action_dict = {
                        'action_type': action_enum.value,
                        'parameters': np.array([0.8, 0.0, 0.5])
                    }
                    obs, reward, terminated, truncated, info = self.sim_env.step(action_dict)
                    state_after = self.sim_env.state
                    
                    if terminated or truncated:
                        if info.get('fell', False):
                            execution_success = False
                            if verbose:
                                print("FAILED (fell)")
                            break
                else:
                    # Execute on real robot
                    action_cmd = ActionCommand(action_enum, 0.8, 0.0, 0.5)
                    motion_cmd = action_cmd.to_motion_command()
                    
                    if motion_cmd['is_gesture']:
                        self.robot_interface.execute_gesture(motion_cmd['gesture_id'])
                    else:
                        self.robot_interface.send_velocity(motion_cmd['velocity'])
                    
                    time.sleep(2.0)  # Wait for action to complete
                    state_after = self.robot_interface.get_state()
                
                if verbose:
                    print("✓")
            
            if verbose and execution_success:
                print(f"   Execution complete!")
        
        except Exception as e:
            execution_success = False
            if verbose:
                print(f"\n   ⚠️  Execution error: {e}")
        
        # Step 4: Evaluate with RLHF
        if verbose:
            print(f"\n4️⃣  Evaluating performance...")
        
        feedback = self.rlhf_model.evaluate_action_sequence(
            command=command,
            actions=action_plan.actions,
            robot_state_before=robot_state,
            robot_state_after=state_after,
            execution_success=execution_success
        )
        
        # Step 5: Log execution for future learning
        if RAG_AVAILABLE:
            self.knowledge_base.add_execution_log(
                command=command,
                actions=action_plan.actions,
                outcome=feedback.feedback_text,
                success=execution_success and feedback.reward_score > 0
            )
        
        # Update metrics
        if execution_success and feedback.reward_score > 0:
            self.success_count += 1
        
        # Store in history
        self.execution_history.append({
            "command": command,
            "plan": action_plan,
            "state_before": robot_state,
            "state_after": state_after,
            "feedback": feedback,
            "success": execution_success,
            "timestamp": datetime.now().isoformat()
        })
        
        # Results
        result = {
            "success": execution_success and feedback.reward_score > 0,
            "plan": action_plan,
            "feedback": feedback,
            "state_before": robot_state,
            "state_after": state_after
        }
        
        if verbose:
            print(f"\n5️⃣  Results:")
            print(f"   Status: {'✓ Success' if result['success'] else '✗ Failed'}")
            print(f"   Success Rate: {self.success_count}/{self.total_commands} ({100*self.success_count/self.total_commands:.1f}%)")
            print(f"   RLHF Reward: {feedback.reward_score:+.3f}")
        
        return result
    
    def export_training_data(self, filepath: str):
        """Export execution history for fine-tuning the LLM agent"""
        training_data = []
        
        for entry in self.execution_history:
            # Format as instruction-following examples
            training_data.append({
                "instruction": entry["command"],
                "output": {
                    "actions": entry["plan"].actions,
                    "reasoning": entry["plan"].reasoning
                },
                "feedback": {
                    "reward": entry["feedback"].reward_score,
                    "success": entry["success"]
                }
            })
        
        with open(filepath, 'w') as f:
            json.dump(training_data, f, indent=2)
        
        print(f"✓ Exported {len(training_data)} training examples to {filepath}")
    
    def get_performance_summary(self) -> Dict:
        """Get summary statistics"""
        if not self.execution_history:
            return {"total": 0}
        
        rewards = [e["feedback"].reward_score for e in self.execution_history]
        safety_scores = [e["feedback"].safety_score for e in self.execution_history]
        
        return {
            "total_commands": self.total_commands,
            "successful": self.success_count,
            "success_rate": self.success_count / self.total_commands if self.total_commands > 0 else 0,
            "avg_reward": np.mean(rewards),
            "avg_safety": np.mean(safety_scores),
            "total_actions_executed": sum(len(e["plan"].actions) for e in self.execution_history)
        }

# Initialize complete LLM-based control system
print("\n" + "="*70)
print("Initializing Complete LLM-Based Robot Control System")
print("="*70 + "\n")

llm_robot_controller = LLMBasedRobotController(
    knowledge_base=knowledge_base if RAG_AVAILABLE else None,
    llm_agent=llm_controller,
    rlhf_model=rlhf_reward_model,
    robot_interface=None  # Use simulation (set to G1RobotInterface for real robot)
)

print("\n✓ System ready for commands!")

### Complete LLM-Based Control System

Now let's integrate all components into a complete system that:
1. Accepts natural language commands
2. Retrieves relevant context from documentation (RAG)
3. Plans action sequences (LLM Agent)
4. Executes actions on robot (existing RL policy)
5. Evaluates performance (RLHF Reward Model)
6. Learns from feedback over time

In [ ]:
@dataclass
class RewardFeedback:
    """Feedback from RLHF reward model"""
    reward_score: float  # -1.0 to 1.0
    safety_score: float
    efficiency_score: float
    goal_progress_score: float
    naturalness_score: float
    feedback_text: str
    improvement_suggestions: List[str]

class RLHFRewardModel:
    """
    LLM-based reward model for Reinforcement Learning from Human Feedback
    
    Provides learned rewards based on:
    - Safety assessment
    - Efficiency evaluation
    - Goal progress estimation
    - Motion naturalness
    """
    
    def __init__(self, 
                 model_name: str = "meta-llama/Llama-3.1-70B-Instruct",
                 enable_feedback_collection: bool = True):
        """
        Initialize RLHF reward model
        
        Args:
            model_name: HuggingFace model for reward evaluation
            enable_feedback_collection: Whether to store feedback for fine-tuning
        """
        self.model_name = model_name
        self.enable_feedback_collection = enable_feedback_collection
        self.feedback_history = []
        
        # Initialize client
        if HF_AVAILABLE:
            hf_token = os.environ.get("HUGGINGFACE_TOKEN")
            if hf_token:
                self.client = InferenceClient(token=hf_token)
                print(f"✓ RLHF Reward Model initialized with {model_name}")
            else:
                print("⚠️  HUGGINGFACE_TOKEN not set")
                self.client = None
        else:
            self.client = None
            print("⚠️  HuggingFace not available")
        
        # Reward model system prompt
        self.system_prompt = """You are an expert evaluator for humanoid robot actions.

Your task is to assess the quality of robot action sequences based on multiple criteria.

Evaluation criteria:
1. Safety (0-1): Risk of falling, collision, or hardware damage
2. Efficiency (0-1): Energy usage, time to complete, smoothness
3. Goal Progress (0-1): How much the action advances toward the objective
4. Naturalness (0-1): How human-like and graceful the motion appears

For each action sequence, provide:
- Individual scores (0-1) for each criterion
- Overall reward score (-1 to +1)
- Textual feedback explaining your evaluation
- Specific improvement suggestions

Output format (JSON):
{
    "safety_score": 0.0-1.0,
    "efficiency_score": 0.0-1.0,
    "goal_progress_score": 0.0-1.0,
    "naturalness_score": 0.0-1.0,
    "reward_score": -1.0 to +1.0,
    "feedback_text": "explanation",
    "improvement_suggestions": ["suggestion1", "suggestion2"]
}

Be strict but fair. Penalize unsafe actions heavily.
"""
    
    def evaluate_action_sequence(self,
                                 command: str,
                                 actions: List[str],
                                 robot_state_before: RobotState,
                                 robot_state_after: RobotState,
                                 execution_success: bool = True) -> RewardFeedback:
        """
        Evaluate an executed action sequence
        
        Args:
            command: Original natural language command
            actions: Sequence of actions executed
            robot_state_before: State before execution
            robot_state_after: State after execution
            execution_success: Whether execution completed without errors
        
        Returns:
            reward_feedback: Detailed evaluation and reward
        """
        if self.client is None:
            # Fallback: simple heuristic reward
            return self._fallback_reward(actions, robot_state_before, robot_state_after, execution_success)
        
        # Build evaluation prompt
        state_delta = self._compute_state_delta(robot_state_before, robot_state_after)
        
        user_prompt = f"""Evaluate this robot action sequence:

Command: "{command}"
Actions executed: {' → '.join(actions)}
Execution status: {"Success" if execution_success else "Failed"}

State changes:
- Goal distance: {robot_state_before.goal_distance:.2f}m → {robot_state_after.goal_distance:.2f}m (Δ: {state_delta['goal_progress']:.2f}m)
- Orientation change: {np.rad2deg(state_delta['orientation_change']):.1f}°
- Base tilt: {np.rad2deg(robot_state_after.base_orientation[:2]):.1f}°
- Obstacle proximity: {np.min(robot_state_after.obstacle_proximity):.2f}m

Provide your evaluation:"""
        
        try:
            response = self.client.chat_completion(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                model=self.model_name,
                max_tokens=400,
                temperature=0.2
            )
            
            # Parse response
            response_text = response.choices[0].message.content
            
            # Extract JSON
            json_match = re.search(r'```json\n(.*?)\n```', response_text, re.DOTALL)
            if json_match:
                response_text = json_match.group(1)
            elif '```' in response_text:
                json_match = re.search(r'```\n(.*?)\n```', response_text, re.DOTALL)
                if json_match:
                    response_text = json_match.group(1)
            
            eval_data = json.loads(response_text)
            
            feedback = RewardFeedback(
                reward_score=eval_data.get("reward_score", 0.0),
                safety_score=eval_data.get("safety_score", 0.5),
                efficiency_score=eval_data.get("efficiency_score", 0.5),
                goal_progress_score=eval_data.get("goal_progress_score", 0.5),
                naturalness_score=eval_data.get("naturalness_score", 0.5),
                feedback_text=eval_data.get("feedback_text", "No feedback"),
                improvement_suggestions=eval_data.get("improvement_suggestions", [])
            )
            
            # Store for potential fine-tuning
            if self.enable_feedback_collection:
                self.feedback_history.append({
                    "command": command,
                    "actions": actions,
                    "state_delta": state_delta,
                    "feedback": feedback,
                    "timestamp": datetime.now().isoformat()
                })
            
            print(f"\n📊 RLHF Evaluation:")
            print(f"   Overall Reward: {feedback.reward_score:+.3f}")
            print(f"   Safety: {feedback.safety_score:.2f} | Efficiency: {feedback.efficiency_score:.2f}")
            print(f"   Goal Progress: {feedback.goal_progress_score:.2f} | Naturalness: {feedback.naturalness_score:.2f}")
            print(f"   Feedback: {feedback.feedback_text}")
            
            return feedback
            
        except Exception as e:
            print(f"⚠️  RLHF evaluation failed: {e}")
            print("   Using fallback reward")
            return self._fallback_reward(actions, robot_state_before, robot_state_after, execution_success)
    
    def _compute_state_delta(self, state_before: RobotState, state_after: RobotState) -> Dict:
        """Compute key state changes"""
        return {
            "goal_progress": state_before.goal_distance - state_after.goal_distance,
            "orientation_change": np.linalg.norm(state_after.base_orientation - state_before.base_orientation),
            "velocity_change": np.linalg.norm(state_after.base_linear_velocity - state_before.base_linear_velocity),
            "stability_change": np.abs(state_after.base_orientation[:2]).mean() - np.abs(state_before.base_orientation[:2]).mean()
        }
    
    def _fallback_reward(self, actions: List[str], 
                        state_before: RobotState, 
                        state_after: RobotState,
                        execution_success: bool) -> RewardFeedback:
        """Simple heuristic reward when LLM unavailable"""
        
        # Safety: penalize large tilts
        tilt = np.abs(state_after.base_orientation[:2]).max()
        safety_score = max(0, 1.0 - tilt / 0.5)  # 0.5 rad = ~30 degrees
        
        # Efficiency: penalize many actions
        efficiency_score = max(0, 1.0 - len(actions) / 10.0)
        
        # Goal progress
        progress = state_before.goal_distance - state_after.goal_distance
        goal_progress_score = np.clip(progress / 1.0, -1, 1)  # Normalize by 1m
        
        # Naturalness: constant (can't assess without LLM)
        naturalness_score = 0.7
        
        # Overall reward (weighted average)
        reward_score = (
            0.4 * safety_score +
            0.2 * efficiency_score +
            0.3 * goal_progress_score +
            0.1 * naturalness_score
        ) * (1.0 if execution_success else 0.5)  # Halve reward if failed
        
        return RewardFeedback(
            reward_score=reward_score,
            safety_score=safety_score,
            efficiency_score=efficiency_score,
            goal_progress_score=goal_progress_score,
            naturalness_score=naturalness_score,
            feedback_text="Heuristic reward (LLM unavailable)",
            improvement_suggestions=[]
        )
    
    def export_feedback_dataset(self, filepath: str):
        """Export collected feedback for fine-tuning reward model"""
        if not self.feedback_history:
            print("⚠️  No feedback collected yet")
            return
        
        with open(filepath, 'w') as f:
            json.dump(self.feedback_history, f, indent=2)
        
        print(f"✓ Exported {len(self.feedback_history)} feedback samples to {filepath}")

# Initialize RLHF reward model
rlhf_reward_model = RLHFRewardModel(
    model_name="meta-llama/Llama-3.1-70B-Instruct",
    enable_feedback_collection=True
)

# Test reward evaluation
print("\n" + "="*60)
print("Testing RLHF Reward Model")
print("="*60)

# Simulate execution
test_command = "Walk forward and wave"
test_actions = ["walk_forward", "idle", "wave"]

state_before = RobotState(
    base_orientation=np.array([0.05, 0.02, 0.0]),
    base_angular_velocity=np.zeros(3),
    base_linear_velocity=np.zeros(3),
    joint_positions=np.zeros(29),
    joint_velocities=np.zeros(29),
    foot_contacts=np.array([1, 1]),
    goal_direction=np.array([1, 0, 0]),
    goal_distance=5.0,
    obstacle_proximity=np.ones(8) * 10.0,
    cmd_velocity=np.zeros(3),
    cmd_mode=1,
    cmd_gesture=0
)

state_after = RobotState(
    base_orientation=np.array([0.03, 0.01, 0.0]),
    base_angular_velocity=np.zeros(3),
    base_linear_velocity=np.zeros(3),
    joint_positions=np.zeros(29),
    joint_velocities=np.zeros(29),
    foot_contacts=np.array([1, 1]),
    goal_direction=np.array([1, 0, 0]),
    goal_distance=4.2,  # Made progress
    obstacle_proximity=np.ones(8) * 10.0,
    cmd_velocity=np.zeros(3),
    cmd_mode=1,
    cmd_gesture=0
)

feedback = rlhf_reward_model.evaluate_action_sequence(
    command=test_command,
    actions=test_actions,
    robot_state_before=state_before,
    robot_state_after=state_after,
    execution_success=True
)

print(f"\nSuggestions:")
for i, suggestion in enumerate(feedback.improvement_suggestions):
    print(f"  {i+1}. {suggestion}")

### RLHF Reward Model

The reward model is a separate LLM that evaluates the quality of actions based on:
- **Safety:** No dangerous movements or instability risks
- **Efficiency:** Minimal energy consumption and time
- **Goal Achievement:** How well the action advances toward the objective
- **Naturalness:** How human-like and smooth the motion is

This provides a learned reward signal that complements the hand-crafted rewards from Section 5.

In [ ]:
@dataclass
class ActionPlan:
    """Planned action sequence from LLM"""
    actions: List[str]
    reasoning: str
    safety_check: bool
    estimated_duration: float
    confidence: float

class LLMAgentController:
    """
    LLM-based high-level controller for natural language robot control
    
    Uses HuggingFace inference API for:
    - Task understanding and decomposition
    - Action sequence planning
    - Safety validation
    """
    
    def __init__(self, 
                 model_name: str = "meta-llama/Llama-3.1-70B-Instruct",
                 knowledge_base: Optional[RobotKnowledgeBase] = None,
                 use_rag: bool = True):
        """
        Initialize LLM controller
        
        Args:
            model_name: HuggingFace model ID (alternatives: mistralai/Mixtral-8x7B-Instruct-v0.1,
                       microsoft/Phi-3-medium-4k-instruct)
            knowledge_base: RAG knowledge base for context retrieval
            use_rag: Whether to use RAG for enhanced context
        """
        self.model_name = model_name
        self.knowledge_base = knowledge_base
        self.use_rag = use_rag and knowledge_base is not None
        
        # Initialize HuggingFace client
        if HF_AVAILABLE:
            hf_token = os.environ.get("HUGGINGFACE_TOKEN")
            if hf_token:
                self.client = InferenceClient(token=hf_token)
                print(f"✓ LLM Controller initialized with {model_name}")
            else:
                print("⚠️  HUGGINGFACE_TOKEN not set. Set it in environment or notebook.")
                print("   Get token from: https://huggingface.co/settings/tokens")
                self.client = None
        else:
            self.client = None
            print("⚠️  HuggingFace not available")
        
        # Available actions (from our RL system)
        self.available_actions = [
            "idle", "walk_forward", "walk_backward", 
            "strafe_left", "strafe_right",
            "rotate_left", "rotate_right",
            "wave", "handshake", "clap"
        ]
        
        # System prompt for robot control
        self.system_prompt = f"""You are an expert robot control assistant for the Unitree G1 humanoid robot.

Your task is to translate natural language commands into safe, executable action sequences.

Available actions: {', '.join(self.available_actions)}

For each command, you must:
1. Understand the user's intent
2. Decompose into a sequence of primitive actions
3. Validate safety (no dangerous movements, respect limits)
4. Provide clear reasoning for your plan

Output format (JSON):
{{
    "actions": ["action1", "action2", ...],
    "reasoning": "explanation of your plan",
    "safety_check": true/false,
    "estimated_duration": seconds,
    "confidence": 0.0-1.0
}}

Safety rules:
- Never suggest actions that could cause falling or collision
- Respect velocity limits: vx[-0.5,0.8], vy[-0.4,0.4], vyaw[-1.0,1.0] m/s
- Always check for obstacles before movement
- Include 'idle' action to stop safely between major transitions
"""
    
    def plan_actions(self, command: str, 
                    robot_state: Optional[RobotState] = None) -> ActionPlan:
        """
        Plan action sequence from natural language command
        
        Args:
            command: Natural language instruction
            robot_state: Current robot state (optional, for context)
        
        Returns:
            action_plan: Planned actions with reasoning
        """
        if self.client is None:
            # Fallback: simple keyword-based parsing
            return self._fallback_planner(command)
        
        # Build context with RAG
        context = ""
        if self.use_rag:
            relevant_docs = self.knowledge_base.retrieve_relevant_context(command, k=3)
            if relevant_docs:
                context = "\n\nRelevant documentation:\n"
                for doc in relevant_docs:
                    context += f"- [{doc['type']}] {doc['text'][:200]}...\n"
        
        # Build user prompt
        state_info = ""
        if robot_state:
            state_info = f"""
Current robot state:
- Position relative to goal: {robot_state.goal_distance:.2f}m
- Orientation: {np.rad2deg(robot_state.base_orientation).tolist()}°
- Obstacles nearby: {np.min(robot_state.obstacle_proximity):.2f}m
"""
        
        user_prompt = f"""Command: {command}
{state_info}
{context}

Plan the action sequence:"""
        
        # Query LLM
        try:
            response = self.client.chat_completion(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                model=self.model_name,
                max_tokens=500,
                temperature=0.3  # Lower temperature for more deterministic planning
            )
            
            # Parse response
            response_text = response.choices[0].message.content
            
            # Extract JSON (handle markdown code blocks)
            json_match = re.search(r'```json\n(.*?)\n```', response_text, re.DOTALL)
            if json_match:
                response_text = json_match.group(1)
            elif '```' in response_text:
                # Try without json tag
                json_match = re.search(r'```\n(.*?)\n```', response_text, re.DOTALL)
                if json_match:
                    response_text = json_match.group(1)
            
            plan_data = json.loads(response_text)
            
            # Validate actions
            valid_actions = [a for a in plan_data["actions"] if a in self.available_actions]
            
            action_plan = ActionPlan(
                actions=valid_actions,
                reasoning=plan_data.get("reasoning", "No reasoning provided"),
                safety_check=plan_data.get("safety_check", False),
                estimated_duration=plan_data.get("estimated_duration", len(valid_actions) * 2.0),
                confidence=plan_data.get("confidence", 0.5)
            )
            
            print(f"\n🤖 LLM Plan:")
            print(f"   Actions: {' → '.join(action_plan.actions)}")
            print(f"   Reasoning: {action_plan.reasoning}")
            print(f"   Safety: {'✓' if action_plan.safety_check else '✗'}")
            print(f"   Confidence: {action_plan.confidence:.2f}")
            
            return action_plan
            
        except Exception as e:
            print(f"⚠️  LLM planning failed: {e}")
            print("   Falling back to rule-based planner")
            return self._fallback_planner(command)
    
    def _fallback_planner(self, command: str) -> ActionPlan:
        """
        Simple keyword-based fallback planner
        (used when LLM unavailable or fails)
        """
        command_lower = command.lower()
        actions = []
        
        # Pattern matching
        if "wave" in command_lower or "hello" in command_lower:
            actions = ["wave"]
        elif "handshake" in command_lower or "shake" in command_lower:
            actions = ["handshake"]
        elif "clap" in command_lower:
            actions = ["clap"]
        elif "forward" in command_lower:
            actions = ["walk_forward", "idle"]
        elif "backward" in command_lower or "back" in command_lower:
            actions = ["walk_backward", "idle"]
        elif "left" in command_lower and "turn" in command_lower:
            actions = ["rotate_left", "idle"]
        elif "right" in command_lower and "turn" in command_lower:
            actions = ["rotate_right", "idle"]
        else:
            actions = ["idle"]  # Default: do nothing if unclear
        
        return ActionPlan(
            actions=actions,
            reasoning="Fallback rule-based matching",
            safety_check=True,
            estimated_duration=len(actions) * 2.0,
            confidence=0.3
        )

# Initialize LLM controller
llm_controller = LLMAgentController(
    model_name="meta-llama/Llama-3.1-70B-Instruct",  # Or use smaller model
    knowledge_base=knowledge_base if RAG_AVAILABLE else None,
    use_rag=True
)

# Test planning (will use fallback if HF token not set)
test_commands = [
    "Walk forward and wave hello",
    "Turn around and return",
    "Greet the visitor with a handshake"
]

print("\n" + "="*60)
print("Testing LLM Action Planning")
print("="*60)

for cmd in test_commands:
    print(f"\n📝 Command: \"{cmd}\"")
    plan = llm_controller.plan_actions(cmd)
    print()

### LLM Agent Controller with HuggingFace

The LLM Agent translates natural language commands into actionable robot instructions using HuggingFace's inference API.

In [ ]:
class RobotKnowledgeBase:
    """
    RAG-based knowledge retrieval system for robot control
    
    Stores and retrieves relevant documentation, code examples,
    and past execution logs to assist LLM decision-making.
    """
    
    def __init__(self, persist_directory: str = "./robot_knowledge_db"):
        self.persist_directory = persist_directory
        self.embeddings = None
        self.vectorstore = None
        self.documents = []
        
        if RAG_AVAILABLE:
            # Initialize embedding model
            self.embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
            
            # Initialize vector database
            self._load_or_create_vectorstore()
            
            print("✓ Knowledge base initialized")
        else:
            print("⚠️  RAG not available - knowledge base disabled")
    
    def _load_or_create_vectorstore(self):
        """Load existing vectorstore or create new one"""
        try:
            self.vectorstore = Chroma(
                persist_directory=self.persist_directory,
                embedding_function=self.embeddings
            )
            print(f"  Loaded {self.vectorstore._collection.count()} documents")
        except:
            self.vectorstore = Chroma(
                persist_directory=self.persist_directory,
                embedding_function=self.embeddings
            )
            print("  Created new vectorstore")
    
    def add_manual_documentation(self, text: str, source: str):
        """
        Add documentation from user manuals
        
        Args:
            text: Documentation content
            source: Source identifier (e.g., "G1_UserManual_v2.1")
        """
        if not RAG_AVAILABLE:
            return
        
        # Split into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        chunks = text_splitter.split_text(text)
        
        # Add metadata
        metadatas = [{"source": source, "type": "manual"} for _ in chunks]
        
        # Add to vectorstore
        self.vectorstore.add_texts(texts=chunks, metadatas=metadatas)
        self.vectorstore.persist()
        
        print(f"✓ Added {len(chunks)} chunks from {source}")
    
    def add_github_code(self, code: str, repo: str, file_path: str):
        """
        Add code examples from GitHub repositories
        
        Args:
            code: Source code content
            repo: Repository name
            file_path: File path in repo
        """
        if not RAG_AVAILABLE:
            return
        
        # Split code into logical chunks (functions, classes)
        chunks = self._split_code(code)
        
        # Add metadata
        metadatas = [{
            "source": f"{repo}/{file_path}",
            "type": "code",
            "repo": repo
        } for _ in chunks]
        
        # Add to vectorstore
        self.vectorstore.add_texts(texts=chunks, metadatas=metadatas)
        self.vectorstore.persist()
        
        print(f"✓ Added {len(chunks)} code snippets from {repo}/{file_path}")
    
    def add_api_documentation(self, api_docs: Dict[str, str]):
        """
        Add SDK API documentation
        
        Args:
            api_docs: Dictionary of {function_name: documentation}
        """
        if not RAG_AVAILABLE:
            return
        
        texts = []
        metadatas = []
        
        for func_name, doc in api_docs.items():
            texts.append(f"Function: {func_name}\n\n{doc}")
            metadatas.append({
                "source": "SDK_API_Reference",
                "type": "api",
                "function": func_name
            })
        
        self.vectorstore.add_texts(texts=texts, metadatas=metadatas)
        self.vectorstore.persist()
        
        print(f"✓ Added {len(texts)} API documentation entries")
    
    def add_execution_log(self, command: str, actions: List[str], 
                         outcome: str, success: bool):
        """
        Log a past execution for future reference
        
        Args:
            command: Natural language command
            actions: Sequence of actions taken
            outcome: Result description
            success: Whether execution succeeded
        """
        if not RAG_AVAILABLE:
            return
        
        log_text = f"""
Command: {command}
Actions: {' -> '.join(actions)}
Outcome: {outcome}
Success: {success}
Timestamp: {datetime.now().isoformat()}
"""
        
        metadata = {
            "source": "execution_log",
            "type": "execution",
            "success": success,
            "timestamp": datetime.now().isoformat()
        }
        
        self.vectorstore.add_texts(texts=[log_text], metadatas=[metadata])
        self.vectorstore.persist()
    
    def retrieve_relevant_context(self, query: str, k: int = 5) -> List[Dict[str, str]]:
        """
        Retrieve most relevant documentation for a query
        
        Args:
            query: Search query (e.g., user command)
            k: Number of results to return
        
        Returns:
            contexts: List of {text, source, type} dictionaries
        """
        if not RAG_AVAILABLE or self.vectorstore is None:
            return []
        
        # Similarity search
        results = self.vectorstore.similarity_search_with_score(query, k=k)
        
        contexts = []
        for doc, score in results:
            contexts.append({
                "text": doc.page_content,
                "source": doc.metadata.get("source", "unknown"),
                "type": doc.metadata.get("type", "unknown"),
                "relevance_score": float(1 - score)  # Convert distance to similarity
            })
        
        return contexts
    
    def _split_code(self, code: str) -> List[str]:
        """Split code into logical chunks (functions/classes)"""
        # Simple regex-based splitting
        # In production, use proper AST parsing
        chunks = []
        
        # Split by function definitions
        pattern = r'(def\s+\w+.*?(?=\ndef\s+|\nclass\s+|\Z))'
        matches = re.findall(pattern, code, re.DOTALL)
        
        if matches:
            chunks.extend(matches)
        else:
            # Fallback: split by lines
            lines = code.split('\n')
            chunk_size = 20
            for i in range(0, len(lines), chunk_size):
                chunks.append('\n'.join(lines[i:i+chunk_size]))
        
        return [c.strip() for c in chunks if c.strip()]

# Initialize knowledge base
knowledge_base = RobotKnowledgeBase()

# Add example documentation
if RAG_AVAILABLE:
    # Example: User manual excerpt
    manual_text = """
    Unitree G1 Locomotion Control:
    
    The G1 robot supports multiple locomotion modes:
    - Stand: Robot maintains upright posture with damping
    - Walk: Controlled walking with velocity commands (vx, vy, vyaw)
    - Balance: Active balance control for external disturbances
    
    Velocity Commands:
    - vx: Forward/backward velocity in m/s (range: -0.5 to 0.8)
    - vy: Left/right strafe velocity in m/s (range: -0.4 to 0.4)
    - vyaw: Rotation velocity in rad/s (range: -1.0 to 1.0)
    
    Safety Limits:
    - Maximum pitch/roll angle: 30 degrees
    - Emergency stop: Available via controller or software command
    - Obstacle detection: Uses depth cameras with 0.3m minimum distance
    """
    
    knowledge_base.add_manual_documentation(manual_text, "G1_UserManual_v2.1")
    
    # Example: API documentation
    api_docs = {
        "Move(vx, vy, vyaw)": "Send velocity command to robot. Returns success status.",
        "StandUp()": "Transition from sitting to standing posture. Duration ~3 seconds.",
        "Damp()": "Enter damping mode (emergency stop). Robot becomes compliant.",
        "ExecuteGesture(gesture_id)": "Perform pre-defined arm gesture. IDs: 0=wave, 1=handshake, 2=clap"
    }
    
    knowledge_base.add_api_documentation(api_docs)
    
    # Example: Code snippet
    code_example = """
def navigate_to_goal(robot, goal_position):
    '''Navigate robot to goal using simple proportional control'''
    current_pos = robot.get_position()
    while distance(current_pos, goal_position) > 0.5:
        # Calculate direction
        dx = goal_position[0] - current_pos[0]
        dy = goal_position[1] - current_pos[1]
        
        # Send velocity command
        robot.Move(dx * 0.3, dy * 0.3, 0.0)
        time.sleep(0.1)
        current_pos = robot.get_position()
    
    robot.Move(0, 0, 0)  # Stop at goal
"""
    
    knowledge_base.add_github_code(code_example, "unitree/examples", "navigation.py")
    
    # Example: Past execution
    knowledge_base.add_execution_log(
        command="Walk forward and wave",
        actions=["walk_forward", "stop", "wave"],
        outcome="Successfully completed greeting gesture",
        success=True
    )
    
    print("\n✓ Example knowledge added to database")

# Test retrieval
if RAG_AVAILABLE:
    print("\nTesting knowledge retrieval...")
    query = "How do I make the robot wave?"
    contexts = knowledge_base.retrieve_relevant_context(query, k=3)
    
    print(f"\nQuery: '{query}'")
    print(f"Retrieved {len(contexts)} relevant contexts:\n")
    for i, ctx in enumerate(contexts):
        print(f"{i+1}. [{ctx['type']}] {ctx['source']}")
        print(f"   Relevance: {ctx['relevance_score']:.3f}")
        print(f"   {ctx['text'][:100]}...")
        print()

### RAG System for Robot Documentation

The RAG (Retrieval-Augmented Generation) system provides the LLM with contextual knowledge from:
- **User Manuals:** Unitree G1 operation guides
- **GitHub Repositories:** Code examples and best practices
- **API Documentation:** SDK function references
- **Past Executions:** Successful task completions and failure cases

In [ ]:
# Install LLM and RAG dependencies
!pip install huggingface_hub transformers sentence-transformers faiss-cpu langchain chromadb
!pip install openai  # For alternative API access

# Set your HuggingFace token (get from https://huggingface.co/settings/tokens)
# You can use inference API (free tier) or serverless endpoints
import os
# os.environ["HUGGINGFACE_TOKEN"] = "hf_xxxxxxxxxxxxxxxxxxxxx"  # Uncomment and add your token

# Imports for LLM control
try:
    from huggingface_hub import InferenceClient
    import transformers
    HF_AVAILABLE = True
except ImportError:
    print("Warning: HuggingFace libraries not available")
    HF_AVAILABLE = False

try:
    from sentence_transformers import SentenceTransformer
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.vectorstores import Chroma
    from langchain.embeddings import HuggingFaceEmbeddings
    RAG_AVAILABLE = True
except ImportError:
    print("Warning: RAG libraries not available")
    RAG_AVAILABLE = False

import re
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass, field
from datetime import datetime

print(f"✓ LLM Setup Complete")
print(f"  - HuggingFace: {HF_AVAILABLE}")
print(f"  - RAG System: {RAG_AVAILABLE}")

### LLM Dependencies & Setup

---
<a id='llm-control'></a>
## 10. LLM-Based Agentic Control with RLHF

This section implements a **language model-based controller** that enables high-level task planning and execution through natural language. The system combines:

1. **LLM Agent Controller:** Uses HuggingFace models to interpret commands and plan actions
2. **RLHF Reward Model:** Secondary model evaluates action quality for reward shaping
3. **RAG System:** Retrieval-Augmented Generation from robot documentation, manuals, and repositories
4. **Fine-tuned Commands:** Domain-specific instruction tuning for robot control

### Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                    LLM-Based Control System                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Natural Language Command: "Walk to the door and wave hello"   │
│            ↓                                                    │
│  ┌──────────────────────────────────────────────────────┐     │
│  │  LLM Agent Controller (HuggingFace Inference API)     │     │
│  │  - Task decomposition                                 │     │
│  │  - Action sequence planning                           │     │
│  │  - Safety validation                                  │     │
│  └────────────────┬─────────────────────────────────────┘     │
│                   │                                             │
│         ┌─────────┴─────────┐                                  │
│         │  RAG System        │                                  │
│         │  - User manuals    │                                  │
│         │  - GitHub repos    │                                  │
│         │  - API docs        │                                  │
│         │  - Past executions │                                  │
│         └─────────┬─────────┘                                  │
│                   │                                             │
│         Action Sequence: [walk_forward, stop, wave]            │
│            ↓                                                    │
│  ┌──────────────────────────────────────────────────────┐     │
│  │  RLHF Reward Model (Evaluator LLM)                   │     │
│  │  - Evaluates action appropriateness                  │     │
│  │  - Scores safety and efficiency                      │     │
│  │  - Provides feedback signal                          │     │
│  └────────────────┬─────────────────────────────────────┘     │
│                   │                                             │
│         Reward Score + Feedback                                │
│            ↓                                                    │
│  ┌──────────────────────────────────────────────────────┐     │
│  │  Motion Execution (Existing RL Policy)               │     │
│  │  - Executes primitive actions                        │     │
│  │  - Monitors state                                    │     │
│  │  - Returns outcome                                   │     │
│  └──────────────────────────────────────────────────────┘     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Benefits

- **Natural Language Interface:** Non-experts can control robot with plain English
- **Contextual Understanding:** RAG provides domain knowledge from manuals and code
- **Adaptive Learning:** RLHF continuously improves action selection from feedback
- **Safe Execution:** LLM validates plans before execution
- **Explainable:** Agent provides reasoning for its decisions